# Análisis de Series Temporales

## Visualización e Intuición

La herramienta principal para el análisis de series temporales es la inspección visual, para ello resulta imprescindible manejar herramientas gráficas. En este curso utilizaremos la librería `matplotlib`. Para leer archivos lo haremos utilizando la librería `csv`.

Para abrir un archivo utilzaremos la sentencia `open` con el handler `with`.

In [1]:
import csv
import matplotlib.pyplot as plt

Siempre hay que conocer la forma que tiene el archivo a leer. En nuestro caso examinaremos dos de ellos:
1. Precios de mercado de Google
```
Date,Open,High,Low,Close,Adj Close,Volume
2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028
```
2. Datos del SMN 
```
FECHA     HORA  TEMP   HUM   PNM    DD    FF     NOMBRE                                             
         [HOA]  [�C]   [%]  [hPa]  [gr] [km/hr]                                                     
02062023     0  18.0   84  1016.4   50   11     AEROPARQUE AERO                                     
02062023     1  18.4   83  1015.5   20    9     AEROPARQUE AERO                                     
02062023     2  18.3   83  1015.2   20    9     AEROPARQUE AERO                                     
02062023     3  18.4   83  1015.2  360   13     AEROPARQUE AERO                                     
02062023     4  18.2   84  1014.6  350    9     AEROPARQUE AERO                                     
```

In [2]:
data_google = '../Data/googleStock/Google_Stock_Train (2010-2022).csv'
data_smn = '../Data/junio-SMN/horario/datohorario20230602.txt'

In [3]:
with open( data_google, 'r') as fp:
    reader = csv.DictReader(fp, delimiter=',')  # Leer como diccionarios
    g_data = []                                 # Lista de diccionarios
    for row in reader:
        g_data.append(row)
for k,v in g_data[1].items():
    print(f'{k}: {v}')

Date: 2010-01-05
Open: 15.695195
High: 15.711712
Low: 15.554054
Close: 15.615365
Adj Close: 15.615365
Volume: 120067812


In [4]:
# Los datos del SMN tienen dos columnas de encabezado, hay que leer con cuidado, 
# parece estar hecho en un encoding traido de windows.
# Los archivos parecen tener ancho de columnas fijas
with open(data_smn, 'rb') as fp:
    headers = next(fp).decode().split()
    units = next(fp).decode(errors='replace').split()
    smn_data = []
    for line in fp:
        lines = line.decode(errors='replace')
        smn_data.append([
            lines[:8].strip(), 
            lines[8:14].strip(), 
            lines[14:20].strip(), 
            lines[20:25].strip(), 
            lines[25:33].strip(), 
            lines[33:38].strip(),
            lines[38:43].strip(),
            lines[43:].strip()
            ])

# limpiando las columnas con salto de carro
clean = []
for l in smn_data:
    if l[0] != '':
        clean.append(l)

for l in range(len(clean)):
    for it in range(len(clean[l])):
        if clean[l][it] == '':
            clean[l][it] = '-1'


smn_data = clean
print(headers)
print(units)
import random
for i in range(50):
    print(random.choice(smn_data))

['FECHA', 'HORA', 'TEMP', 'HUM', 'PNM', 'DD', 'FF', 'NOMBRE']
['[HOA]', '[�C]', '[%]', '[hPa]', '[gr]', '[km/hr]']
['02062023', '16', '26.4', '55', '1010.5', '50', '20', 'CORRIENTES AERO']
['02062023', '11', '13.6', '71', '1008.5', '160', '15', 'SAN RAFAEL AERO']
['02062023', '6', '14.8', '89', '1006.3', '360', '6', 'CORDOBA AERO']
['02062023', '18', '24.5', '66', '1010.8', '50', '19', 'FORMOSA AERO']
['02062023', '6', '12.2', '57', '1001.1', '320', '26', 'TRELEW AERO']
['02062023', '2', '10.9', '50', '988.4', '340', '44', 'USHUAIA AERO']
['02062023', '13', '10.0', '59', '1429.0', '270', '24', 'MAQUINCHAO']
['02062023', '19', '19.1', '58', '1010.6', '200', '9', 'MENDOZA AERO']
['02062023', '11', '13.5', '96', '1013.1', '40', '7', 'TANDIL AERO']
['02062023', '10', '19.8', '81', '1013.7', '360', '19', 'MORON AERO']
['02062023', '2', '19.0', '75', '1009.1', '40', '9', 'CORDOBA OBSERVATORIO']
['02062023', '15', '19.3', '73', '1008.5', '270', '9', 'CORONEL SUAREZ AERO']
['02062023', '11', '

Utilizando la libreria `datetime` para manejar datos de fechas y horas.

En este caso se convertirá el dato de fechas a objetos de datetime

In [17]:
from datetime import datetime, timedelta

In [24]:
smn_data_standarized = [{}]
for d in smn_data:
    # Since data is not very well structured and a lot of carriage return appear
    if len(d) > 7:
        smn_data_standarized.append({
            'fecha': datetime.strptime(d[0],'%d%m%Y')+ timedelta(hours= int(d[1])),
            'temp': float(d[2]),
            'hum': float(d[3]),
            'pres': float(d[4]),
            'v_direcc': int(d[5]),
            'v_intens': float(d[6]),
            'estac': d[7]
        }
        )
smn_data_standarized = smn_data_standarized[1:]

## Analizando las estaciones meteorologicas

In [25]:
estaciones = []
for element in smn_data_standarized:
    estaciones.append(element['estac'])
estaciones = set(estaciones)
print(f'Existe un total de {len(estaciones)} estaciones')

for e in estaciones:
    print(e)


Existe un total de 121 estaciones
VILLA GESELL AERO
PERITO MORENO AERO
BARILOCHE AERO
SANTA ROSA DE CONLARA AERO
MARCOS JUAREZ AERO
SAN LUIS AERO
EL PALOMAR AERO
TRENQUE LAUQUEN
CHEPES
VILLA DOLORES AERO
COMODORO RIVADAVIA AERO
BASE ORCADAS
OLAVARRIA AERO
CHAMICAL AERO
USHUAIA AERO
PEHUAJO AERO
BENITO JUAREZ AERO
BUENOS AIRES OBSERVATORIO
PIGUE AERO
MARIANO MORENO AERO
AZUL AERO
LA RIOJA AERO
TANDIL AERO
TINOGASTA
CORRIENTES AERO
GUALEGUAYCHU AERO
IGUAZU AERO
SANTA ROSA AERO
JUJUY U N
GOBERNADOR GREGORES AERO
CHAPELCO AERO
RIO CUARTO AERO
CORDOBA OBSERVATORIO
CORONEL SUAREZ AERO
CATAMARCA AERO
AEROPARQUE AERO
METAN
JUJUY AERO
MERLO AERO
BAHIA BLANCA AERO
BASE SAN MARTIN
ITUZAINGO
ESCUELA DE AVIACION MILITA
MENDOZA AERO
PILAR OBSERVATORIO
SUNCHALES AERO
EL BOLSON AERO
RESISTENCIA AERO
NUEVE DE JULIO
VILLA REYNOLDS AERO
TRELEW AERO
USPALLATA
ESQUEL AERO
PUERTO MADRYN AERO
RIO GALLEGOS AERO
VICTORICA
LAS FLORES
CORDOBA AERO
PASO DE INDIOS
TERMAS DE RIO HONDO AERO
LABOULAYE AERO
SALTA AERO

# Filtrando los datos para una estación particular

In [26]:
estacion = 'SALTA'
data_salta = []
for element in smn_data_standarized:
    if element['estac'].find(estacion) >= 0:
        data_salta.append(element)
data_salta

[{'fecha': datetime.datetime(2023, 6, 2, 0, 0),
  'temp': 15.2,
  'hum': 83.0,
  'pres': 1511.0,
  'v_direcc': 270,
  'v_intens': 4.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 2, 1, 0),
  'temp': 15.1,
  'hum': 84.0,
  'pres': 1507.0,
  'v_direcc': 200,
  'v_intens': 4.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 2, 2, 0),
  'temp': 15.0,
  'hum': 83.0,
  'pres': 1503.0,
  'v_direcc': 180,
  'v_intens': 4.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 2, 3, 0),
  'temp': 15.1,
  'hum': 82.0,
  'pres': 1498.0,
  'v_direcc': 230,
  'v_intens': 6.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 2, 4, 0),
  'temp': 14.4,
  'hum': 86.0,
  'pres': 1493.0,
  'v_direcc': 0,
  'v_intens': 0.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 2, 5, 0),
  'temp': 13.9,
  'hum': 87.0,
  'pres': 1487.0,
  'v_direcc': 0,
  'v_intens': 0.0,
  'estac': 'SALTA AERO'},
 {'fecha': datetime.datetime(2023, 6, 